In [21]:
import os
import json
import pandas as pd
from env import *

In [22]:
directory_path = json_path
output_path = xlsx_path+'urls.xlsx'

In [23]:
def merge_json_files_into_dataframe(directory_path):
    all_files = os.listdir(directory_path)
    json_files = [f for f in all_files if f.endswith('.json')]
    
    merged_df = pd.DataFrame()
    
    for json_file in json_files:
        file_path = os.path.join(directory_path, json_file)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            # json field name
            if "URLs" in data:
                temp_df = pd.DataFrame(data["URLs"], columns=["URLs"])
                merged_df = pd.concat([merged_df, temp_df], axis=0)
    
    merged_df.reset_index(drop=True, inplace=True)
    return merged_df

In [24]:
merged_dataframe = merge_json_files_into_dataframe(directory_path)
# File extension extract
# merged_dataframe['Extension'] = merged_dataframe['File_Names'].str.extract(r'(\.\w+)$')
# merged_dataframe['Extension'] = merged_dataframe['Extension'].str.lower()

# Domain extract
merged_dataframe['Domain'] = merged_dataframe['URLs'].str.extract(r'https?://([^/]+)')
merged_dataframe.head()  # Displaying the first few rows of the merged DataFrame

,URLs,Domain
0,https://portal.msrc.microsoft.com/en-us/securi...,portal.msrc.microsoft.com
1,https://threatrecon.nshc.net/2019/04/30/sector...,threatrecon.nshc.net
2,https://docs.microsoft.com/en-us/windows/deskt...,docs.microsoft.com
3,https://www.cvedetails.com/vulnerability-list/...,www.cvedetails.com
4,https://blog.trendmicro.com/trendlabs-security...,blog.trendmicro.com


In [25]:
extension_stats = merged_dataframe['Domain'].value_counts().reset_index()
extension_stats.columns = ['Domain', 'Count']
extension_stats.head()

,Domain,Count
0,twitter.com,120
1,www.clearskysec.com,105
2,www.fireeye.com,103
3,unit42.paloaltonetworks.com,103
4,blog.trendmicro.com,94


In [26]:
with pd.ExcelWriter(output_path) as writer:
    merged_dataframe.to_excel(writer, sheet_name='dataframe', index=False)
    extension_stats.to_excel(writer, sheet_name='stats', index=False)